# Load implicit dataset → build CSR matrix

- Dùng MovieLens 100k (ratings → implicit interaction).
- Note: CSR matrix là Compressed Sparse Row matrix – một cách lưu trữ và tính toán ma trận thưa (sparse matrix) cực kỳ quan trọng trong recommender systems (chỉ lưu những ô khác 0)

In [31]:
import numpy as np
import pandas as pd
from collections import defaultdict
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import random

In [32]:
# Load MovieLens 100k
cols = ["user", "item", "rating", "timestamp"]
df = pd.read_csv(
    "/Users/codexplore/Developer/repos/recommender-system/data/movielens/ml-100k/u.data",
    sep="\t", names=cols
)


In [33]:
# Implicit feedback
df["interaction"] = (df["rating"] >= 4).astype(int)

# Encode to 0-index
user2id = {u:i for i,u in enumerate(df["user"].unique())}
item2id = {i:j for j,i in enumerate(df["item"].unique())}

df["uid"] = df["user"].map(user2id)
df["iid"] = df["item"].map(item2id)

In [34]:
n_users = df["uid"].nunique()
n_items = df["iid"].nunique()

# Build CSR matrix
R = csr_matrix(
    (df["interaction"], (df["uid"], df["iid"])),
    shape=(n_users, n_items)
)
print("Number of users:", n_users)
print("Number of items:", n_items)
print("CSR shape:", R.shape)
print("df's shape: ", df.shape)

df.head()

Number of users: 943
Number of items: 1682
CSR shape: (943, 1682)
df's shape:  (100000, 7)


,user,item,rating,timestamp,interaction,uid,iid
0,196,242,3,881250949,0,0,0
1,186,302,3,891717742,0,1,1
2,22,377,1,878887116,0,2,2
3,244,51,2,880606923,0,3,3
4,166,346,1,886397596,0,4,4


# Sessionization + time decay score

Giả lập session + decay theo thời gian (để giải thích recency bias).

$$
\text{decay}(t) = \exp\left(-\frac{t_{\max} - t}{\text{HALF\_LIFE}}\right)
$$

$$
\text{weighted\_interaction} = \text{interaction} \times \text{decay}(t)
$$

Where:
- $t$ is the event timestamp,
- $t_{\max}$ is the latest timestamp in the dataset,
- $\text{HALF\_LIFE}$ is the decay time constant (here, 30 days in seconds).


In [35]:
df = df.sort_values(["uid", "timestamp"])

SESSION_GAP = 30 * 60  # 30 minutes * 60 seconds as the timestamp is in seconds

df["prev_ts"] = df.groupby("uid")["timestamp"].shift(1)
# Mark a new session if the gap is > 30 minutes or if it’s the first event for that user.
df["new_session"] = (df["timestamp"] - df["prev_ts"] > SESSION_GAP) | df["prev_ts"].isna()
# Increment a session counter each time a new session is detected.
df["session_id"] = df.groupby("uid")["new_session"].cumsum()

# Time decay
max_ts = df["timestamp"].max()
HALF_LIFE = 30 * (24 * 3600)  # 30 days * 24 hours * 3600 seconds

df["decay"] = np.exp(-(max_ts - df["timestamp"]) / HALF_LIFE)
# Compute a recency weight: newer events are closer to 1, older events decay exponentially.
df["weighted_interaction"] = df["interaction"] * df["decay"]

df.head()


,user,item,rating,timestamp,interaction,uid,iid,prev_ts,new_session,session_id,decay,weighted_interaction
0,196,242,3,881250949,0,0,0,NaN,True,1,0.009625,0.000000
13733,196,286,5,881250949,1,0,289,881250949.0,False,1,0.009625,0.009625
78787,196,269,3,881250949,0,0,491,881250949.0,False,1,0.009625,0.000000
6910,196,306,4,881251021,1,0,380,881250949.0,False,1,0.009625,0.009625
25726,196,340,3,881251045,0,0,751,881251021.0,False,1,0.009625,0.000000


# Train/Test split (Leave-One-Out, temporal)

In [36]:
df_train = []
df_test = []

for uid, hist in df.groupby("uid"):
    hist = hist.sort_values("timestamp")
    if len(hist) < 2:
        continue
    df_train.append(hist.iloc[:-1])
    df_test.append(hist.iloc[-1:])

df_train = pd.concat(df_train)
df_test = pd.concat(df_test)

# Train CSR
R_train = csr_matrix(
    (df_train["interaction"], (df_train["uid"], df_train["iid"])),
    shape=(n_users, n_items)
)


# Baseline Models

## Metrics: Recall@K, NDCG@K

In [ ]:
def recall_at_k(gt_item, recs, k):
    return int(gt_item in recs[:k])

def ndcg_at_k(gt_item, recs, k):
    if gt_item in recs[:k]:
        # +1 as array indices are 0‑based, but the NDCG formula uses 1‑based rank 
        rank = np.where(recs[:k] == gt_item)[0][0] + 1
        return 1 / np.log2(rank + 1)
    return 0


## Precompute Similarities

In [38]:
# User-user similarity
user_sim = cosine_similarity(R_train)

# Item-item similarity
item_sim = cosine_similarity(R_train.T)


In [39]:
user_sim.shape, user_sim[:5]

((943, 943),
 array([[1.        , 0.03109852, 0.09981303, ..., 0.04545455, 0.        ,
         0.05170877],
        [0.03109852, 1.        , 0.10243324, ..., 0.09329556, 0.        ,
         0.14150983],
        [0.09981303, 0.10243324, 1.        , ..., 0.02495326, 0.03701166,
         0.11354659],
        [0.15462183, 0.07052482, 0.17919744, ..., 0.05154061, 0.02548236,
         0.09772039],
        [0.        , 0.09225312, 0.03701166, ..., 0.06741999, 0.3       ,
         0.153393  ]]))

In [40]:
item_sim.shape, item_sim[:5]

((1682, 1682),
 array([[1.        , 0.27658655, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.27658655, 1.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
         0.        ],
        [0.06320606, 0.09724333, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.14887604, 0.36647628, 0.        , ..., 0.        , 0.        ,
         0.        ]]))

## Random

In [41]:
def recommend_random(user_id, topk=10):
    return np.random.choice(n_items, size=topk, replace=False)


## Popularity

In [ ]:
# Popularity-based recommendation
# R_train.sum(axis=0): total interactions per item in the training set
item_pop = np.array(R_train.sum(axis=0)).flatten()
# gives item indices sorted by popularity, descending (most popular first).
pop_rank = np.argsort(item_pop)[::-1]

def recommend_popularity(user_id, topk=10):
    return pop_rank[:topk]


## Content-based RecSys

### Load item metadata (GENRES)

In [43]:
item_cols = [
    "item_id", "title", "release_date", "video_release",
    "imdb_url", "unknown", "Action", "Adventure", "Animation",
    "Children", "Comedy", "Crime", "Documentary", "Drama",
    "Fantasy", "FilmNoir", "Horror", "Musical", "Mystery",
    "Romance", "SciFi", "Thriller", "War", "Western"
]

items = pd.read_csv(
    "https://files.grouplens.org/datasets/movielens/ml-100k/u.item",
    sep="|", names=item_cols, encoding="latin-1"
)

items["iid"] = items["item_id"].map(item2id)
items = items.dropna(subset=["iid"])
items["iid"] = items["iid"].astype(int)

genre_cols = item_cols[5:]
print(len(genre_cols), genre_cols)

items.head()


19 ['unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'FilmNoir', 'Horror', 'Musical', 'Mystery', 'Romance', 'SciFi', 'Thriller', 'War', 'Western']


,item_id,title,release_date,video_release,imdb_url,unknown,Action,Adventure,Animation,Children,...,FilmNoir,Horror,Musical,Mystery,Romance,SciFi,Thriller,War,Western,iid
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,24
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,147
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,233
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,47
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,75


### Build TF-IDF item vectors from genres
$$
\text{tf\_idf}(t,d) = \text{tf}(t,d) \times \log\left(\frac{N}{1 + \text{df}(t)}\right)
$$

Where:
- (t) = term, (d) = document,
- (N) = total number of documents,
- $\text{df}(t)$ = number of documents containing (t).

Example: 
- Suppose you have 3 items with `genre_text`:
    - Item A: "Action Comedy"
    - Item B: "Action Thriller"
    - Item C: "Drama"
- Documents = 3.
- Term counts (TF):
    - In Item A: Action=1, Comedy=1
    - In Item B: Action=1, Thriller=1
    - In Item C: Drama=1
- Document frequency (DF):
    - "Action" appears in 2 docs
    - "Comedy" in 1
    - "Thriller" in 1
    - "Drama" in 1
- IDF (roughly):
    - "Action" gets lower IDF because it appears in 2/3 docs.
    - "Comedy/Thriller/Drama" get higher IDF because they appear in only 1 doc.
- So TF‑IDF vectors look like:
    - Item A: high weight for Comedy, lower for Action
    - Item B: high weight for Thriller, lower for Action
    - Item C: high weight for Drama

In [74]:
# collects the genre column names where the value is 1 and joins them into a space‑separated string (e.g., "Action Comedy")
items["genre_text"] = items[genre_cols].apply(
    lambda x: " ".join([g for g,v in x.items() if v == 1]),
    axis=1
)

tfidf = TfidfVectorizer()
# Converts those genre strings into TF‑IDF vectors (sparse matrix). Each column corresponds to a genre term.
item_tfidf = tfidf.fit_transform(items["genre_text"])

vocab_size = len(tfidf.vocabulary_)
print("TF-IDF vocabulary size:", vocab_size)
print("TF-IDF feature names:", item_tfidf.shape)
# Prepares a dense feature matrix sized by total items × number of TF‑IDF features.
item_feat = np.zeros((item_tfidf.shape[0], item_tfidf.shape[1]))
item_feat[items["iid"].values] = item_tfidf.toarray()

iid = 1
print(items[items["iid"]==iid]["genre_text"].values)
print(item_feat[iid])


TF-IDF vocabulary size: 19
TF-IDF feature names: (1682, 19)
['Crime FilmNoir Mystery Thriller']
[0.         0.         0.         0.         0.         0.4522705
 0.         0.         0.         0.6320217  0.         0.
 0.52182997 0.         0.         0.3517008  0.         0.
 0.        ]


### Content-based Recommender

In [ ]:
def recommend_content_based(uid, k=10):
    user_items = R_train[uid].indices
    if len(user_items) == 0:
        return recommend_popularity(uid, k)
    
    # It builds a user’s content profile by averaging the item feature vectors of the items they interacted with.
    user_profile = item_feat[user_items].mean(axis=0, keepdims=True)
    scores = cosine_similarity(user_profile, item_feat).flatten()

    scores[user_items] = -1  # Exclude items the user has already interacted with
    return np.argsort(scores)[::-1][:k]


In [77]:
alpha = 0.7  # content weight

def recommend_content_hybrid(user_id, topk=10):
    user_items = R_train[user_id].indices
    if len(user_items) == 0:
        return recommend_popularity(user_id, topk)

    user_profile = item_feat[user_items].mean(axis=0)
    content_score = item_feat @ user_profile
    pop_score = item_pop / item_pop.max()

    scores = alpha * content_score + (1 - alpha) * pop_score
    scores[user_items] = -1
    return np.argsort(scores)[::-1][:topk]


## User-based CF (manual cosine)

In [ ]:
def recommend_user_cf(user_id, topk=10):
    # Pull the similarity vector between the target user and all users.
    sims = user_sim[user_id]
    
    # scores reflect what similar users liked (weighted by similarity)
    scores = sims @ R_train.toarray()

    seen_items = R_train[user_id].indices
    scores[seen_items] = -1

    return np.argsort(scores)[::-1][:topk]


In [48]:
sims = user_sim[0]
scores = sims @ R_train.toarray()

sims.shape, R_train.toarray().shape, scores.shape, scores

((943,),
 (943, 1682),
 (1682,),
 array([ 6.81857224, 14.79474924,  0.05288859, ...,  0.        ,
         0.        ,  0.        ]))

## Item-based CF (manual cosine)

In [49]:
def recommend_item_cf(user_id, topk=10):
    user_items = R_train[user_id].indices
    scores = np.zeros(n_items)

    for i in user_items:
        scores += item_sim[i]

    scores[user_items] = -1
    return np.argsort(scores)[::-1][:topk]


# Evaluation

In [50]:
K = 10
results = defaultdict(list)

models = {
    "Random ": recommend_random,
    "Popularity ": recommend_popularity,
    "Content based": recommend_content_based,
    "Content based + Popularity": recommend_content_hybrid,
    "UserCF ": recommend_user_cf,
    "ItemCF ": recommend_item_cf
}

for _, row in df_test.iterrows():
    uid = row["uid"]
    gt_item = row["iid"]

    for name, model in models.items():
        recs = model(uid, K)
        results[name].append({
            "recall": recall_at_k(gt_item, recs, K),
            "ndcg": ndcg_at_k(gt_item, recs, K)
        })

for name, vals in results.items():
    recall = np.mean([v["recall"] for v in vals])
    ndcg = np.mean([v["ndcg"] for v in vals])
    print(f"{name:12s} | Recall@{K}: {recall:.4f} | NDCG@{K}: {ndcg:.4f}")


Random       | Recall@10: 0.0074 | NDCG@10: 0.0031
Popularity   | Recall@10: 0.0435 | NDCG@10: 0.0199
Content based | Recall@10: 0.0117 | NDCG@10: 0.0061
Content based + Popularity | Recall@10: 0.0668 | NDCG@10: 0.0352
UserCF       | Recall@10: 0.0923 | NDCG@10: 0.0481
ItemCF       | Recall@10: 0.0965 | NDCG@10: 0.0524
